[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hosein9574/My-agents/blob/main/first%20agent%20(togetherai).ipynb)


In [ ]:
# Install required libraries
!pip install --quiet openai gradio

# Imports
import openai
import gradio as gr
from getpass import getpass

# 🔑 Get Together.ai API key securely
openai.api_key = getpass("key")
base_url = "https://api.together.xyz/v1"
model = "meta-llama/Llama-3-8b-chat-hf"

# 🎯 System prompt and database schema
context = [
    {'role': 'system', 'content': """
You are a bot to assist in creating SQL commands.
All your answers should start with:
"This is your SQL:", followed by an SQL query to fulfill the user's request.

Your SQL Database is composed of the following tables:
Keep the SQL simple and clean.
Explain briefly how the SQL works after the query.
If the question cannot be answered with the DB, reply with:
"Sorry, I can only help with SQL-related database questions."
"""},

    {'role': 'system', 'content': """
First table:
{
  "tableName": "employees",
  "fields": [
    { "nombre": "ID_usr", "tipo": "int" },
    { "nombre": "name", "tipo": "string" }
  ]
}
"""},

    {'role': 'system', 'content': """
Second table:
{
  "tableName": "salary",
  "fields": [
    { "nombre": "ID_usr", "type": "int" },
    { "name": "year", "type": "date" },
    { "name": "salary", "type": "float" }
  ]
}
"""},

    {'role': 'system', 'content': """
Third table:
{
  "tablename": "studies",
  "fields": [
    { "name": "ID", "type": "int" },
    { "name": "ID_usr", "type": "int" },
    { "name": "educational level", "type": "int" },
    { "name": "Institution", "type": "string" },
    { "name": "Years", "type": "date" },
    { "name": "Speciality", "type": "string" }
  ]
}
"""}
]

# 🔁 Function to send messages to Together.ai using OpenAI-compatible API
def continue_conversation(messages, temperature=0):
    client = openai.OpenAI(api_key=openai.api_key, base_url=base_url)
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content


# 🧠 Chat handler
def gradio_chat(message, history):
    chat_history = context.copy()
    for user, assistant in history:
        chat_history.append({"role": "user", "content": user})
        chat_history.append({"role": "assistant", "content": assistant})
    chat_history.append({"role": "user", "content": message})
    chat_history.append({'role': 'system', 'content': """
Only return SQL Orders.
If you can't return an SQL order, say sorry and ask for another.
"""})
    return continue_conversation(chat_history)


# UI Components
InputText = gr.Textbox(label="SQL Request", info="Ask something the database can answer", scale=6)
examples = [
    "Who is the highest-paid employee?",
    "How many employees with degrees do we have?"
]


# 🚀 Launch Gradio app (FIXED)
gr.ChatInterface(
    fn=gradio_chat,
    textbox=InputText,  # ✅ Use 'textbox' not 'inputs'
    title="SQL Generator using LLaMA 3 (Together.ai)",
    examples=examples,
    submit_btn="Get My SQL"
).launch()
print(1)